# Using protein language models, Foldseek, and Domainator to identify divergent restriction enzymes

Requires esmologs to be installed in the same environment as Domainator. See [here](../../docs/esm_3b_foldseek.md) for more information.

![Overview of RE discovery](../media/RE_workflow_web.png)

__References__:

- Johnson, Sean R., Meghana Peshwa, and Zhiyi Sun. “Sensitive Remote Homology Search by Local Alignment of Small Positional Embeddings from Protein Language Models.” eLife 12 (October 27, 2023). [https://doi.org/10.7554/eLife.91415.1](https://doi.org/10.7554/eLife.91415.1).
- Roberts, Richard J, Tamas Vincze, Janos Posfai, and Dana Macelis. “REBASE: A Database for DNA Restriction and Modification: Enzymes, Genes and Genomes.” Nucleic Acids Research 51, no. D1 (January 6, 2023): D629–30. [https://doi.org/10.1093/nar/gkac975](https://doi.org/10.1093/nar/gkac975).
- Kempen, Michel van, Stephanie S. Kim, Charlotte Tumescheit, Milot Mirdita, Jeongjae Lee, Cameron L. M. Gilchrist, Johannes Söding, and Martin Steinegger. “Fast and Accurate Protein Structure Search with Foldseek.” Nature Biotechnology, May 8, 2023. [https://doi.org/10.1038/s41587-023-01773-0](https://doi.org/10.1038/s41587-023-01773-0).



In [ ]:
from esmologs.utils import iter_fasta
from domainator.partition_seqfile import partition_seqfile
import torch
import subprocess
import pandas as pd

In [2]:
%%bash

mkdir -p ../resources

#download ESM-3 3B 3Di weights
if [ ! -f ../resources/ESM-2_3B_3Di.pt ]; then
    wget -O ../resources/ESM-2_3B_3Di.pt https://zenodo.org/record/8174960/files/ESM-2_3B_3Di.pt --no-check-certificate > /dev/null 2>&1
fi



In [ ]:
%%bash
if [ ! -f ../resources/ncbi_complete.gb ]; then
    # For the paper, we did not limit --num_recs, but that will give a file larger than 360 Gb, so we limit to 100 records here
    # just comment out the --num_recs option to download the complete dataset
    domainator_db_download.py --db ncbi_complete_genome_proks -o ../resources/ncbi_complete.gb --gene_call unannotated --num_recs 100
fi


In [4]:
# REBASE gold standard proteins in this example were downloaded on Dec. 12, 2022

# extract methyltransferase sequences from REBASE Gold Standards

prefixes = {"M.", "M1.", "M2.", "M3.", "M4."} #headers for things that are MTs
with open("../resources/REBASE_Gold_Standards.fasta", "r") as pep_in, open("REBASE_gold_MTs.fasta", "w") as MT_out:
    for pep_header, pep_seq in iter_fasta(pep_in, full_name=True):
       pep_name = pep_header.split(' ')[0]
       if any((pep_name.startswith(prefix) for prefix in prefixes)):
           print(f">{pep_header}\n{pep_seq}",file=MT_out)

In [5]:
# extract restriction enzyme sequences from REBASE Gold Standards
prefixes = {"M.", "M1.", "M2.", "M3.", "M4.", "S.", "H.", "C.", "S1.", "S2."} #headers for things that are not REs
descriptions = {"enzyme/methyltransferase"}

with open("../resources/REBASE_Gold_Standards.fasta", "r") as pep_in, open("REBASE_gold_REs.txt", "w") as RE_out:
    for pep_header, pep_seq in iter_fasta(pep_in, full_name=True):
        pep_name = pep_header.split(' ')[0]

        if any((pep_name.startswith(prefix) for prefix in prefixes)):
            continue

        if any((desc in pep_header for desc in descriptions)):
            continue

        print(pep_name,file=RE_out)

In [ ]:
# extract Type II restriction enzyme sequences from REBASE Gold Standards
# type_II_subset.txt is a file with the names of the Type II REs, it excludes a few, such as TspDTI and BbrUII, which seem to be far overrepresented in hits, and may be lead to lots of false positives.
! select_by_contig.py -i ../resources/REBASE_Gold_Standards.fasta -o Type_II_RE.fasta --fasta_out --contigs_file ../resources/type_II_subset.txt


In [ ]:
# Convert the Type II RE sequences to 3Di
! predict_from_ESM2_to_3Di.py -i Type_II_RE.fasta -o Type_II_RE.3di.fasta --weights ../resources/ESM-2_3B_3Di.pt --device cuda:0

In [ ]:
# Convert the Type II RE sequences to a Foldseek database
! fasta2foldseek.py --aa Type_II_RE.fasta --tdi Type_II_RE.3di.fasta -o Type_II_RE

In [11]:
# extract neighborhoods 4 ORFs up and downstream from MTs
! domain_search.py -i ../resources/ncbi_complete.gb -r REBASE_gold_MTs.fasta --no_annotations -o MT_nbs.gb --cds_range 4 -e 1e-10 --max_region_overlap 0.1 --cpu 0  > /dev/null 2>&1

In [15]:
# add REBASE gold standard phmmer annotations
! domainate.py -i MT_nbs.gb -r ../resources/REBASE_Gold_Standards.fasta -o MT_nbs.annot.gb -e 0.001 --cpu 0  --max_overlap 0.6 > /dev/null 2>&1


In [ ]:
# get contigs not containing any RE annotations
! select_by_contig.py -i MT_nbs.annot.gb -o orphan_MTs.gb --invert --domains_file REBASE_gold_REs.txt

In [ ]:
# Run domainate with protein language model and Foldseek

# if multiple GPUs are available, partition the file and run domainate.py on each partition
num_gpus = torch.cuda.device_count()
_, offset_list = partition_seqfile("orphan_MTs.gb", partitions=num_gpus)
open_processes = []
for i, (fname, offset, recs_to_read) in enumerate(offset_list):
    open_processes.append(
    subprocess.Popen(["domainate.py", "-i", fname, "--offset", str(offset), "--recs_to_read", str(recs_to_read),
                    "--foldseek", "Type_II_RE",
                    "--hits_only", 
                    "-o", f"cryptic_RMs_{i}.gb",
                    "--cpu", "2",
                    "-e", "1e-10",
                    "--max_overlap", "0.6", 
                    "--esm2_3Di_device", f"cuda:{i}",
                    "--esm2_3Di_weights", "../resources/ESM-2_3B_3Di.pt"])
    )

for p in open_processes:
    p.wait()
    
# merge the results
! cat cryptic_RMs_*.gb > cryptic_RMs.gb


In [ ]:
# Extract the peptides from the cryptic RMs
! extract_peptides.py -i cryptic_RMs.gb -o putative_REs.gb --domains_file ../resources/type_II_subset.txt --database Type_II_RE

In [ ]:
# extract and combine cryptic RM annotations
! enum_report.py -i cryptic_RMs.gb -o cryptic_RMs.tsv --length --domains --domain_descriptions --domain_search --column_names contig 'contig_length(bp)' domains domain_descriptions best_MT MT_description MT_query_start MT_query_end MT_query_length MT_hit_start MT_hit_end MT_hit_strand MT_domain_score 'MT_domain_identity(%)'
! enum_report.py --by cds -i cryptic_RMs.gb -o cryptic_RMs_CDS_to_contig.tsv 
! enum_report.py -i putative_REs.gb -o putative_REs.tsv --length --domains --domain_descriptions --column_names RE_CDS 'RE_length(aa)' RE_domains RE_domain_descriptions



In [38]:
# merge the tables
contig_df = pd.read_csv("cryptic_RMs.tsv", sep="\t")
RM_to_CDS = pd.read_csv("cryptic_RMs_CDS_to_contig.tsv", sep="\t")
RE_df = pd.read_csv("putative_REs.tsv", sep="\t")

# merge the dataframes
RE_df = RE_df.merge(RM_to_CDS, left_on="RE_CDS", right_on="cds", how="left")
RE_df = RE_df.merge(contig_df, left_on="contig", right_on="contig", how="left")

RE_df.drop_duplicates(subset="RE_CDS", keep='first', inplace=True, ignore_index=True)

RE_df.to_csv("RE_metadata.tsv", sep="\t", index=False)

In [ ]:
# Calculate EFI scores between the putative REs
! seq_dist.py --cpu 16 -i putative_REs.gb -r putative_REs.gb --sparse RE_dist.hdf5 --mode efi_score

In [44]:
! matrix_report.py -i RE_dist.hdf5 -o /dev/stdout

/mnt/home/sjohnson/anaconda3/envs/esmologs/bin/matrix_report.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  __import__('pkg_resources').require('domainator==0.6.3')
Matrix Report
Total values: 7333264
Non-zero values: 390995
Mean: 119.22262179927645
Median: 106.64179905430898
Min: 5.870701074551252
Max: 885.9112107407537
        

      --------------------------------------------------------------------------------------------------------
      |                                            Matrix Values                                             |
      --------------------------------------------------------------------------------------------------------

 64765|  o                                                 
 61356|  o                                                 
 57948|  o                                                 
 54539|  o                                                 
 51130|  o   

In [17]:
# build a sequence similarity network. After the SSN is built, the xgmml file can be visualized with Cytoscape. Use the Layout->Prefuse Force Directed Layout to visualize the network.
! build_ssn.py -i RE_dist.hdf5 --xgmml putative_REs.xgmml --metadata RE_metadata.tsv --color_by best_MT --lb 40 --cluster_tsv putative_REs_clusters.tsv

In [20]:
import pandas as pd
import subprocess
RE_df = pd.read_table("RE_metadata.tsv")
clusters_table = pd.read_table("putative_REs_clusters.tsv")
# rename columns
clusters_table.rename(columns={"contig": "RE_CDS"}, inplace=True)
# add cluster column to RE metadata
RE_df = RE_df.merge(clusters_table, left_on="RE_CDS", right_on="RE_CDS", how="left")
for i in range(0, 100):
    idx = i + 1
    idx_zfill = str(idx).zfill(3)
    RE_df[RE_df["cluster"] == idx]["contig"].to_csv(f"cluster_{idx_zfill}.tsv", sep="\t", index=False, header=False)
    subprocess.run(["select_by_contig.py", "-i", "cryptic_RMs.gb", "-o", f"cluster_rep_{idx_zfill}.gb", "--contigs_file", f"cluster_{idx_zfill}.tsv", "--first", "1"])



In [28]:
import webbrowser
RE_df[["contig", "cluster"]].to_csv("clusters_for_plots.tsv", sep="\t", index=False)
! plot_contigs.py -i cluster_rep_*.gb --html clusters_plot.html --metadata clusters_for_plots.tsv --color_by "cluster"
webbrowser.open("clusters_plot.html")

True